In [9]:
Web_page_search_context = "novartis Iptacopan competitors"
number_of_pages = 2
chunk_size = 1000
chunk_overlap =20
docs_to_vectorize = 1000
embeddings_to_use ='Ollama'
model_to_use = 'Ollama'

from pathlib import Path
import os
import yaml
from pathlib import Path 
from box.exceptions import BoxValueError


CUSTOM_ENV_PATH = lambda a : '/'.join(os.get_exec_path()[0].split('\\')[:-2])+'/'+a
CUSTOM_ENV_NAME = CUSTOM_ENV_PATH('chatter.env')

def extract_links ( ac, link_in_dict, dict_key ='link' ):
    """
    reads a list of dictionaries and returns specific value for a key 
    """
    link_in_list = [ link_in_dict[i][dict_key] for i in range(len(link_in_dict)) ]
    print ( "Activity ", ac, ": Done: extracting webpage from dict")
    return link_in_list


In [12]:
min (docs_to_vectorize, len())

10

In [5]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(CUSTOM_ENV_NAME)

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")

from langchain_community.utilities import GoogleSearchAPIWrapper


def extract_related_links ():
    """
    read context and number of results from params.yaml
    returns a text file with saved list of web pages to be vectorized
    """
    # extract the context and length of search for websites
    search_context =Web_page_search_context
    search_num = number_of_pages
    print (search_context, "for ", search_num, " pages")

    # use the context and length to do a google search using an api wrapper
    api_wrapper = GoogleSearchAPIWrapper( siterestrict = False )
    link_in_dict = api_wrapper.results (search_context  , search_num )
    print ( "Activity 2: Done: getting webpages from Google wrapper")
    # Extract the links from the above dictionary to a list so as to be used for webbasedloader
    link_in_list =  extract_links( 3,  link_in_dict , 'link')

    return link_in_list

list_link = extract_related_links()

novartis Iptacopan competitors for  2  pages


d:\Study2\git\Environments\novas\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


Activity 2: Done: getting webpages from Google wrapper
Activity  3 : Done: extracting webpage from dict


In [6]:
list_link

['https://www.novartis.com/news/media-releases/novartis-investigational-iptacopan-phase-iii-study-demonstrates-clinically-meaningful-and-highly-statistically-significant-proteinuria-reduction-patients-iga-nephropathy-igan',
 'https://www.biospace.com/article/novartis-pnh-trial-results-put-pressure-on-astrazeneca-s-ultomiris-and-soliris/']

In [12]:
# from langchain_community.document_loaders import WebBaseLoader
# import bs4
# list_link = ["https://www.novartis.com/news/media-releases/novartis-investigational-iptacopan-phase-iii-study-demonstrates-clinically-meaningful-and-highly-statistically-significant-proteinuria-reduction-patients-iga-nephropathy-igan" , "https://www.biospace.com/article/novartis-pnh-trial-results-put-pressure-on-astrazeneca-s-ultomiris-and-soliris/"]
# loader = WebBaseLoader(web_path = list_link,)
#                         # bs_kwargs= dict( parse_only = bs4.SoupStrainer
#                         #                 (class_ = ("post-content","post-title","post-header") )), )
 
# web_docs = loader.load()
# web_docs

In [15]:
import yaml
from pathlib import Path 
import os

from src.constants import *
from src.utils import read_yaml, read_from_text

from dotenv import load_dotenv
load_dotenv(CUSTOM_ENV_NAME)

from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain import hub
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

import streamlit as st
import time

def create_retreiver (links_to_load):
    """
    read chunk size and overlap from params.yaml
    """
    # # read the list of webpage to be taken
    # links_to_load = read_from_text ( 5, WEB_PAGE_TXT )

    # use a loader to get all content
    loader = WebBaseLoader(web_paths = links_to_load,)
                    #        bs_kwargs= dict(parse_only = bs4.SoupStrainer(
                    #        class_ = ("post-title","post-content","post-header")
                    #    )),)    
    web_docs = loader.load()
    print ("size of the web docs : ",len(web_docs))
    print ("Activity ", 6, ": Done: webpage content from links")
    
    # extract the size and overlap chars from param file
    print (chunk_size, "is the chunk size choosen with ", chunk_overlap, " chars overlaps ")

    # splitting the docs based on above size etc
    documents = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap).split_documents(web_docs)
    # taking only top n docs
    docs_to_vectorize_take = min ( docs_to_vectorize, len(web_docs))
    documents_filtered = documents[:docs_to_vectorize_take]
    # choosing the embeddings to use
    if embeddings_to_use == "Instruct":
        print ( "Instruct Embeddings" )
        choosen_embeddings = HuggingFaceInstructEmbeddings(model_name = 'hkunlp/instructor-xl',  
                                                   model_kwargs={"device":"cpu"},
                                                   encode_kwargs ={'normalize_embeddings': True} )
    else :
        choosen_embeddings = OllamaEmbeddings()
        print ( "Ollama Embeddings" )
    # creatign vector store and eventually retreiver
    verctordb = FAISS.from_documents(documents_filtered, choosen_embeddings )
    #retriever = verctordb.as_retriever()
    return verctordb



def get_retrieval_chain (retriever):
    # get prompt
    # prompt = hub.pull("hwchase17/openai-functions-agent")
    # print( prompt.messages )
    prompt = ChatPromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks. Use the provided context only to answer the question. 
    {prompt_examples}
    If you don't know the answer, just say that you don't know. {prompt_text}
    Please provide the most accurate response based on the question.
    <context>
    {context}
    <context>
    Questions: {input}
    """)

    #llm = Ollama(model = "llama2")
    llm= ChatOllama(model="llama2")

    # creating document chain
    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain (retriever, document_chain)
    return retrieval_chain




def talk_to_stream (retrieval_chain):
    st.title("Demo with LLAMA2")

    input_text = st.text_input ("Input the question you have about Iptacopan ")
    prompt_text = " Use three sentences maximum and keep the answer concise. "
    prompt_few_text = """ "Input":"where was Iptacopan discovered?", "Output": "iscovered at the Novartis Biomedical Research, iptacopan is currently in development for a range of complement-mediated diseases including paroxysmal nocturnal hemoglobinuria (PNH), immunoglobulin A nephropathy (IgAN), C3 glomerulopathy (C3G), immune complex membranoproliferative glomerulonephritis (IC-MPGN) and atypical hemolytic uremic syndrome (aHUS)", 
    "Input":"How does Iptacopan compares with Soliris", "Output": "Iptacopan was compared to AstraZeneca/Alexion’s Soliris (eculizumab) and Ultomiris (ravulizumab). Both drugs are anti-complement component 5 (C5) monoclonal antibodies, approved for PNH and atypical hemolytic uremic syndrome, generalized myasthenia gravis and neuromyelitis optica spectrum disorder"."""
    prompt_examples = "Few sample questions and answers are " + prompt_few_text

    #input_text = "what is Iptacopan"
    if input_text:
        start_time = time.process_time()
        response = retrieval_chain.invoke ({"input":input_text, "prompt_text":prompt_text, "prompt_examples": prompt_examples })
        print ("response time: ",time.process_time()-start_time)
        print ( response['answer'] )
        st.write(response['answer'])

In [22]:
prompt_text = " Use three sentences maximum and keep the answer concise. "
prompt_examples = """Few sample questions and answers are 
"Input":"where was Iptacopan discovered?", "Output": "iscovered at the Novartis Biomedical Research, iptacopan is currently in development for a range of complement-mediated diseases including paroxysmal nocturnal hemoglobinuria (PNH), immunoglobulin A nephropathy (IgAN), C3 glomerulopathy (C3G), immune complex membranoproliferative glomerulonephritis (IC-MPGN) and atypical hemolytic uremic syndrome (aHUS)", 
"Input":"How does Iptacopan compares with Soliris", "Output": "Iptacopan was compared to AstraZeneca/Alexion’s Soliris (eculizumab) and Ultomiris (ravulizumab). Both drugs are anti-complement component 5 (C5) monoclonal antibodies, approved for PNH and atypical hemolytic uremic syndrome, generalized myasthenia gravis and neuromyelitis optica spectrum disorder"."""

prompt = ChatPromptTemplate.from_template(
"""
You are an assistant for question-answering tasks. Use the provided context only to answer the question. 
{prompt_examples}
If you don't know the answer, just say that you don't know. {prompt_text}
Please provide the most accurate response based on the question.
<context>
{context}
<context>
Questions: {input}
""")


In [5]:
# retr = create_retreiver (list_link)

size of the web docs :  10
Activity  6 : Done: webpage content from links
1000 is the chunk size choosen with  20  chars overlaps 
Ollama Embeddings


In [16]:
vcdb = create_retreiver (list_link)

size of the web docs :  2
Activity  6 : Done: webpage content from links
1000 is the chunk size choosen with  20  chars overlaps 
Ollama Embeddings


In [14]:
# vcdb.save_local ( os.getcwd() , 'nova_vector_db')

In [20]:
# os.getcwd()

'd:\\Study2\\git\\Leep_Ipta_QA_demo'

In [44]:
# vcdb_load = FAISS.load_local ( 'd:/Study2/git/Leep_Ipta_QA_demo',index_name  = 'nova_vector_db',
#                                embeddings = OllamaEmbeddings() ,allow_dangerous_deserialization = True   ) 

In [25]:
rc  = get_retrieval_chain (vcdb.as_retriever())

In [29]:
input_text = "what are the competitors of Iptacopan"
prompt_text = " Use three sentences maximum and keep the answer concise. "
prompt_few_text = """ "Input":"where was Iptacopan discovered?", "Output": "iscovered at the Novartis Biomedical Research, iptacopan is currently in development for a range of complement-mediated diseases including paroxysmal nocturnal hemoglobinuria (PNH), immunoglobulin A nephropathy (IgAN), C3 glomerulopathy (C3G), immune complex membranoproliferative glomerulonephritis (IC-MPGN) and atypical hemolytic uremic syndrome (aHUS)", 
"Input":"How does Iptacopan compares with Soliris", "Output": "Iptacopan was compared to AstraZeneca/Alexion’s Soliris (eculizumab) and Ultomiris (ravulizumab). Both drugs are anti-complement component 5 (C5) monoclonal antibodies, approved for PNH and atypical hemolytic uremic syndrome, generalized myasthenia gravis and neuromyelitis optica spectrum disorder"."""

prompt_examples = "Few sample questions and answers are " + prompt_few_text



response = rc.invoke ({"input":input_text, "prompt_text":prompt_text, "prompt_examples": prompt_examples })
print ( response['answer'] )

Based on the provided context, the competitors of Iptacopan are:

1. AstraZeneca/Alexion's Soliris (eculizumab) and Ultomiris (ravulizumab). These drugs are also anti-complement component 5 (C5) monoclonal antibodies, approved for PNH and atypical hemolytic uremic syndrome, generalized myasthenia gravis, and neuromyelitis optica spectrum disorder.

Please note that this information is based on the provided context and may not be up-to-date or comprehensive.


# older codes

In [ ]:
def get_retrieval_chain_older (retriever):
    # get prompt
    # prompt = hub.pull("hwchase17/openai-functions-agent")
    # print( prompt.messages )
    prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on the provided context only.
    Please provide the most accurate response based on the question
    <context>
    {context}
    <context>
    Questions: {input}
    """)

    #llm = Ollama(model = "llama2")
    llm= ChatOllama(model="llama2")

    # creating document chain
    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain (retriever, document_chain)
    return retrieval_chain


In [ ]:
def get_retrieval_chain_new (retriever):
    # get prompt
    # prompt = hub.pull("hwchase17/openai-functions-agent")
    # print( prompt.messages )
    few_shot_examples = [{"Input":"where was Iptacopan discovered?", "Output": "iscovered at the Novartis Biomedical Research, iptacopan is currently in development for a range of complement-mediated diseases including paroxysmal nocturnal hemoglobinuria (PNH), immunoglobulin A nephropathy (IgAN), C3 glomerulopathy (C3G), immune complex membranoproliferative glomerulonephritis (IC-MPGN) and atypical hemolytic uremic syndrome (aHUS)"}, 
                         {"Input":"How does Iptacopan compares with Soliris", "Output": "Iptacopan was compared to AstraZeneca/Alexion’s Soliris (eculizumab) and Ultomiris (ravulizumab). Both drugs are anti-complement component 5 (C5) monoclonal antibodies, approved for PNH and atypical hemolytic uremic syndrome, generalized myasthenia gravis and neuromyelitis optica spectrum disorder"}]
    few_shot_template  = ChatPromptTemplate.from_messages(
     [
        ("human", "{input}"),
        ("ai", "{output}")
    ] )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=few_shot_template,
        examples=few_shot_examples,
    )

    #llm = Ollama(model = "llama2")
    llm= ChatOllama(model="llama2")

    # creating document chain
    document_chain = create_stuff_documents_chain(llm, few_shot_prompt)
    retrieval_chain = create_retrieval_chain (retriever, document_chain)
    return retrieval_chain